In [8]:
!pip install yfinance scikit-learn flask flask-ngrok

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
from flask import Flask, render_template_string, request
from flask_ngrok import run_with_ngrok

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

class StockModel:
    def __init__(self, ticker):
        self.ticker = ticker
        self.model = LinearRegression()

    def fetch_data(self, start_date, end_date):
        # Fetch OHLC data for the given date range
        df = yf.download(self.ticker, start=start_date, end=end_date)
        return df

    def preprocess_data(self, df):
        # Feature Engineering: Adding Moving Averages and Target
        df['MA_5'] = df['Close'].rolling(window=5).mean()
        df['MA_10'] = df['Close'].rolling(window=10).mean()
        df['Target'] = df['Close'].shift(-1)  # The next day's closing price
        df.dropna(inplace=True)  # Drop rows with NaN values
        return df

    def train_model(self, df):
        X = df[['MA_5', 'MA_10']]
        y = df['Target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.model.fit(X_train, y_train)
        print(f'Model trained with score: {self.model.score(X_test, y_test)}')

        # Save the model
        with open('stock_model.pkl', 'wb') as f:
            pickle.dump(self.model, f)

    def load_model(self):
        with open('stock_model.pkl', 'rb') as f:
            self.model = pickle.load(f)

    def predict(self, latest_data):
        return self.model.predict(latest_data)

def run_pipeline(ticker, start_date, end_date):
    stock_model = StockModel(ticker)
    df = stock_model.fetch_data(start_date, end_date)
    df = stock_model.preprocess_data(df)
    stock_model.train_model(df)

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    if request.method == 'POST':
        ticker = request.form['ticker']
        start_date = request.form['start_date']
        end_date = request.form['end_date']

        stock_model = StockModel(ticker)
        stock_model.load_model()

        # Fetch the data for the specified date range
        data = stock_model.fetch_data(start_date, end_date)

        if not data.empty:
            # Calculate moving averages
            latest_MA_5 = data['Close'].rolling(window=5).mean().iloc[-1]
            latest_MA_10 = data['Close'].rolling(window=10).mean().iloc[-1]

            # Prepare data for prediction
            latest_input = np.array([[latest_MA_5, latest_MA_10]])

            # Make prediction
            prediction = stock_model.predict(latest_input)[0]
        else:
            prediction = "No data available for the specified date range."

    # HTML template
    html_template = '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Stock Price Prediction</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f4f4f4;
            }
            h1 {
                text-align: center;
                color: #333;
            }
            .container {
                max-width: 600px;
                margin: 0 auto;
                background: white;
                padding: 20px;
                border-radius: 5px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            form {
                display: flex;
                flex-direction: column;
            }
            label {
                margin-bottom: 5px;
                font-weight: bold;
            }
            input[type="text"],
            input[type="date"],
            button {
                padding: 10px;
                margin-bottom: 15px;
                border: 1px solid #ccc;
                border-radius: 5px;
            }
            button {
                background-color: #5cb85c;
                color: white;
                border: none;
                cursor: pointer;
            }
            button:hover {
                background-color: #4cae4c;
            }
            .prediction {
                text-align: center;
                margin-top: 20px;
                font-size: 18px;
                color: #d9534f;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Stock Price Prediction</h1>
            <form method="POST">
                <label for="ticker">Enter Stock Ticker:</label>
                <input type="text" id="ticker" name="ticker" required placeholder="e.g., AAPL">

                <label for="start_date">Start Date (YYYY-MM-DD):</label>
                <input type="date" id="start_date" name="start_date" required>

                <label for="end_date">End Date (YYYY-MM-DD):</label>
                <input type="date" id="end_date" name="end_date" required>

                <button type="submit">Predict</button>
            </form>

            {% if prediction is not none %}
                <div class="prediction">
                    <h2>Predicted Next Closing Price: {{ prediction }}</h2>
                </div>
            {% endif %}
        </div>
    </body>
    </html>
    '''
    return render_template_string(html_template, prediction=prediction)

# Train the model initially (optional)
# run_pipeline('AAPL', '2020-01-01', '2023-01-01')

# Run the Flask app
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll